# 크롤링

## 1) Googlemap 크롤링

### 1. 리뷰 크롤링

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

In [ ]:
def google_reviews(store_list, gu, count):
    chrome_driver_path = "C:/Users/sally/OneDrive/바탕 화면/chromedriver-win64/chromedriver.exe"

    for store in store_list:
        driver = None
        try:
            # 새로운 드라이버 세션 시작
            service = Service(chrome_driver_path)
            options = Options()
            options.add_argument('--start-maximized')
            driver = webdriver.Chrome(service=service, options=options)
            driver.get('https://www.google.co.kr/maps')

            search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
            search_box.clear()
            search_box.send_keys(f'{gu} {store}')
            search_box.send_keys('\n')

            # 리뷰 탭 버튼이 로드될 때까지 대기
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//button[@aria-label="{store} 리뷰"]')))

            try:
                # 리뷰 탭 버튼 클릭
                reviews_tab_button = driver.find_element(By.XPATH, f'//button[@aria-label="{store} 리뷰"]')
                reviews_tab_button.click()
            except Exception as e:
                print(f"Error finding reviews tab button for {store}: {e}")
                continue

            sleep(5)

            # 장소의 별점 영역 클릭
            try:
                rating_section_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[1]'
                rating_section = driver.find_element(By.XPATH, rating_section_xpath)
                actions = ActionChains(driver)
                actions.move_to_element(rating_section).click().perform()
            except Exception as e:
                print(f"Error finding rating section for {store}: {e}")
                continue

            sleep(2)

            reviews = []
            last_length = 0
            scroll_attempts = 0
            empty_review_count = 0
            max_scroll_attempts = 490  # 필요한 경우 조정 가능
            max_empty_reviews = 10  # 비어있는 리뷰 최대 허용 수

            while len(reviews) < count and scroll_attempts < max_scroll_attempts:
                try:
                    # PAGE_DOWN 키를 사용하여 스크롤
                    actions.send_keys('\ue00f').perform()  # \ue00f는 PAGE_DOWN 키
                    sleep(5)  # 로드 시간 충분히 확보

                    # "자세히" 버튼 클릭
                    more_buttons = driver.find_elements(By.XPATH, '//button[@aria-label="더보기" and @class="w8nwRe kyuRq"]')
                    for button in more_buttons:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView();", button)  # 버튼을 화면에 스크롤
                            WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button))  # 버튼이 클릭 가능할 때까지 대기
                            actions.move_to_element(button).click().perform()
                            sleep(1)  # 클릭 후 잠시 대기하여 내용이 확장되도록 함
                        except Exception as e:
                            print(f"Error clicking '자세히' button: {e}")
                            continue

                    # 새로운 리뷰 수집
                    new_reviews = driver.find_elements(By.XPATH, '//span[@class="wiI7pd"]')
                    if len(new_reviews) > last_length:
                        for review in new_reviews[last_length:]:
                            review_text = review.text.strip()
                            if review_text == "":
                                empty_review_count += 1
                            else:
                                empty_review_count = 0  # 리셋 비어있는 리뷰 카운트
                            reviews.append(review_text)
                        last_length = len(new_reviews)
                        scroll_attempts = 0  # 새로운 리뷰가 로드되면 시도 횟수 리셋
                    else:
                        scroll_attempts += 1  # 새로운 리뷰가 없으면 시도 횟수 증가

                    # 비어있는 리뷰가 max_empty_reviews에 도달했을 때 중지
                    if empty_review_count >= max_empty_reviews:
                        print(f"Stopping as {empty_review_count} consecutive empty reviews found for {store}.")
                        break

                    # 새로운 리뷰가 없고 max_scroll_attempts에 도달했을 때 중지
                    if scroll_attempts >= max_scroll_attempts:
                        print(f"Reached max scroll attempts for {store}, stopping search.")
                        break
                except Exception as e:
                    print(f"Error during scrolling and review extraction for {store}: {e}")
                    break

            reviews_text = reviews[:count]
            df = pd.DataFrame(reviews_text, columns=['Review'])
            df.to_csv(f'{store}_reviews.csv', index=False, encoding='utf-8-sig')
            print(f"Saved {len(reviews_text)} reviews for {store} to {store}_reviews.csv")

        except Exception as e:
            print(f"Error processing {store}: {e}")
        
        finally:
            if driver:
                driver.quit()

In [ ]:
# 장소 리스트 (102개)
store_list = [
'83타워'
,'대구수목원'
,'김광석 다시그리기길'
,'두류공원'
,'수성못'
,'앞산공원'
,'디아크문화관'
,'남평문씨 본리세거지'
,'마비정벽화마을'
,'불로동 고분군'
,'국립대구박물관'
,'서문시장'
,'팔공산케이블카'
,'아양기찻길'
,'달성공원'
,'이월드'
,'경상감영공원'
,'송해공원'
,'월광수변공원'
,'이상화고택'
,'강정고령보 고령복'
,'팔공산하늘정원'
,'앞산전망대'
,'스파밸리'
,'국채보상운동기념공원'
,'앞산 케이블카'
,'대구 3.1만세운동길'
,'스파크랜드'
,'동촌유원지'
,'대명유수지'
,'벽화마을(옹기종기행복마을)'
,'대구근대역사관'
,'네이처파크'
,'아르떼 수성랜드'
,'진밭골 산림공원'
,'하중도유채꽃단지'
,'일연공원'
,'대구국제공항'
,'대가야역사테마관광지'
,'대구 팔공산 갓바위'
,'국립대구과학관'
,'월곡역사공원'
,'봉무공원'
,'대구아쿠아리움'
,'영남제일관'
,'도동서원'
,'안지랑곱창골목'
,'침산공원'
,'옻골마을 돌담길'
,'꽃보라동산'
,'수성패밀리파크'
,'대구미술관'
,'아양교'
,'함지근린공원'
,'방짜유기박물관'
,'연암공원'
,'해맞이공원'
,'2·28기념중앙공원'
,'대구예술발전소'
,'망우당공원'
,'금호강 자연생태공원'
,'남지장사'
,'향촌문화관'
,'화산마을'
,'천주교 성모당'
,'군위 아미타여래삼존 석굴'
,'남산어린이공원'
,'학산공원'
,'대구섬유박물관'
,'운문사승가대학'
,'동산계곡'
,'소재사'
,'대구향교'
,'도동 측백나무 숲'
,'대구약령시장'
,'비슬산 자연휴양림'
,'화원자연휴양림'
,'칠성종합시장'
,'노곡교'
,'대구삼성창조캠퍼스'
,'가산산성'
,'수태골'
,'대구약령시 한의약박물관'
,'교동시장'
,'리틀 포레스트 영화 촬영지'
,'송림사'
,'대한불교조계종 선본사'
,'군파크 루지'
,'무주구천동 관광단지',
'주교좌 계산 대성당',
'수창청춘맨숀',
'동화사',
'청라언덕',
'유가사',
'고산골 공룡공원',
'대구시민안전테마파크',
'서남시장',
'아이니 테마파크',
'염매시장',
'이현공원',
'대구문화예술회관',
'롯데아울렛 율하점'
]

#장소별 최대 500개 리뷰 가져오기
google_reviews(store_list, '대구', 500)

### 2. 주소 크롤링

In [ ]:
# 빈 리스트 생성
missing_files = []

for place in places:
    filename = f"{place}_reviews.csv"
    
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        # "주소"와 "장소" 컬럼 추가
        df['주소'] = ''
        df['장소'] = place
        
        # 변경된 데이터프레임을 저장
        df.to_csv(filename, index=False)
    else:
        missing_files.append(place)

# 존재하지 않는 파일 목록 출력
print("파일이 존재하지 않는 장소 목록:")
print(missing_files)

In [ ]:
# 주소가 빈 장소들을 저장할 리스트
places_with_empty_address = []

# 모든 파일을 순회하면서 주소가 빈 행이 있는지 확인
for place in sotre_list:
    file_path = f"{place}_reviews.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if df['주소'].isna().any() or (df['주소'] == '').any():
            places_with_empty_address.append(place)
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")

print("주소가 빈 장소들:", places_with_empty_address)

### 3. 장소별 생성된 파일 하나로 합치기

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# 빈 데이터프레임 생성 (컬럼 이름을 맞추기 위해 첫 번째 파일을 사용)
first_file = f"{store_list[0]}_reviews.csv"
combined_df = pd.read_csv(first_file)

In [ ]:
# 첫 번째 파일을 제외한 나머지 파일들을 결합
for place in store_list[1:]:
    file_path = f"{place}_reviews.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")

# 합쳐진 데이터프레임 저장
combined_df.to_csv("combined_reviews.csv", index=False)

print("모든 CSV 파일이 성공적으로 합쳐졌습니다.")

In [ ]:
c = pd.read_csv('combined_reviews.csv')

c

## 2) 대한민국 구석구석 크롤링

### 1. 모든 관광지 이름 리스트로 만들기

In [ ]:
from selenium.common.exceptions import ElementNotInteractableException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://korean.visitkorea.or.kr/curation/cr_place_list.do?hotPlaceType=Place&regionCode=4')

In [ ]:
# 명시적 대기 설정

wait = WebDriverWait(driver, 10)

# 스크립트 실행 시간 제한 설정

driver.set_script_timeout(200) 

In [ ]:
# '더보기+' 버튼 클릭하여 모든 관광지 로드

while True:
    
    try:
        # 페이지 하단으로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # 스크롤 후 로딩 시간 대기

        # 페이지를 약간 위로 스크롤
        driver.execute_script("window.scrollBy(0, -200);")  
        time.sleep(1)  
        
        # '더보기+' 버튼 찾기
        more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#contents > div > div.hotplace_sub > div.box_wrap > div.list.hotplace_list > div.more > button')))
        more_button.click()
        time.sleep(1) 
        
    except Exception as e:
        print("더 이상 로드할 관광지가 없거나 예외 발생:", e)
        break

In [ ]:
# 다시 페이지 맨 위로 올라가기

driver.execute_script("window.scrollTo(0, 0);")
time.sleep(1)

In [ ]:
# 모든 관광지 이름 요소 가져오기
try:
    # 모든 관광지 이름을 포함하는 strong 태그를 한 번에 가져옵니다.
    place_elements = driver.find_elements(By.CLASS_NAME, 'tit')
    
    # 관광지 이름을 리스트로 저장
    place_names = [place.text for place in place_elements]
    
    # 빈 문자열 제거
    place_names = [name for name in place_names if name.strip()]

    # 마지막 관광지 이름 제거
    if place_names:  
        removed_name = place_names.pop()
    
    # 총 개수 출력
    print(f"총 가져온 관광지 개수: {len(place_names)}")
    
    # 관광지 이름 출력
    for name in place_names:
        print(name)

except Exception as e:
    print("관광지 이름을 찾는 데 실패했습니다. 셀렉터를 확인하거나 대기 시간을 늘려보세요.", e)

### 2. 첫 관광지 상세정보 & 리뷰 크롤링

In [ ]:
# 페이지 맨 위로 스크롤
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(1)

# 첫 번째 관광지 이름 요소 접근
first_place_element = place_elements[0]
                                     
# 관광지 요소 클릭
first_place_element.click()                                    
time.sleep(20) 

# 상세 정보 클릭
detail_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '상세정보')))
detail_button.click()

# 스크롤 하여 상세 정보 보기
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(2)

# 주소, 휴일, 이용시간, 주차 정보 수집
address = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(3) > span').text
holidays = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(5) > span').text
opening_hours = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(4) > span').text
parking = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(6) > span').text

# 여행톡 클릭
travel_talk_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '여행톡')))
travel_talk_button.click()

# 스크롤 및 댓글 더보기 클릭
while True:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        # 페이지를 약간 위로 스크롤
        driver.execute_script("window.scrollBy(0, -1000);")  
        time.sleep(1)  

        more_comments_button = driver.find_element(By.CSS_SELECTOR, '#commentMore > a')
        more_comments_button.click()
        time.sleep(1)
                
    except (NoSuchElementException, ElementNotInteractableException) as e:
        print("더 이상 클릭할 '댓글 더보기' 버튼이 없습니다.")
        break
        
# 댓글 데이터 수집
comments = driver.find_elements(By.CSS_SELECTOR, 'div.list_reply > ul > li') 
    
all_reviews = []
    
for comment in comments:
    
    author = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > em').text
    date = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > span').text
    content = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > p').text

        
    all_reviews.append({
        '관광지 이름': '이월드',
        '주소': address,
        '휴일': holidays,
        '이용시간': opening_hours,
        '주차': parking,
        '댓글 작성자': author,
        '댓글 작성 시간': date,
        '댓글': content
    })

In [ ]:
# 데이터프레임으로 변환
df = pd.DataFrame(all_reviews)
print(df)

# 데이터 저장
df.to_csv('첫번째 관광지.csv', index=False)

driver.quit()

In [ ]:
a = pd.read_csv('./첫번째 관광지.csv')

a

### 3. 전체 관광지 상세정보 & 댓글 크롤링

In [ ]:
# 데이터 저장을 위한 리스트 초기화
all_reviews = []

# 각 관광지에 대해 데이터 수집
for i in range(len(place_names)):
    try:
        # 새로 요소를 찾아 가져오기 (stale 문제 방지)
        place_elements = driver.find_elements(By.CLASS_NAME, 'tit')
        
        # n번째 관광지 요소 접근
        place_element = place_elements[i]
        
        # 관광지 요소 클릭
        place_element.click()
        time.sleep(10)  

        # 상세 정보 클릭
        detail_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '상세정보')))
        detail_button.click()

        # 스크롤 하여 상세 정보 보기
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

        # 주소, 휴일, 이용시간, 주차 정보 수집
        address = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(3) > span').text
        holidays = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(5) > span').text
        opening_hours = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(4) > span').text
        parking = driver.find_element(By.CSS_SELECTOR, '#detailinfoview > div > div.inr_wrap > div > ul > li:nth-child(6) > span').text

        # 여행톡 클릭
        travel_talk_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '여행톡')))
        travel_talk_button.click()

        # 스크롤 및 댓글 더보기 클릭
        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)

                # 페이지를 약간 위로 스크롤
                driver.execute_script("window.scrollBy(0, -1000);")
                time.sleep(1)

                more_comments_button = driver.find_element(By.CSS_SELECTOR, '#commentMore > a')
                more_comments_button.click()
                time.sleep(1)

            except (NoSuchElementException, ElementNotInteractableException):
                print("더 이상 클릭할 '댓글 더보기' 버튼이 없습니다.")
                break

        # 댓글 데이터 수집
        comments = driver.find_elements(By.CSS_SELECTOR, 'div.list_reply > ul > li') 

        for comment in comments:
            try:
                author = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > em').text
                date = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > span').text
                content = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > p').text

                # 리뷰 데이터 추가
                all_reviews.append({
                    '관광지 이름': place_names[i],
                    '주소': address,
                    '휴일': holidays,
                    '이용시간': opening_hours,
                    '주차': parking,
                    '댓글 작성자': author,
                    '댓글 작성 시간': date,
                    '댓글': content
                })
                
            except NoSuchElementException as e:
                print(f"댓글 수집 중 오류 발생: {e}")
                continue

        # 뒤로가기
        driver.back()
        time.sleep(20)

    except Exception as e:
        print(f"오류 발생: {place_names[i]}, Error: {str(e)}")
        break

In [ ]:
# 데이터 저장
if all_reviews:
    df = pd.DataFrame(all_reviews)
    df.to_csv('대구_관광지리뷰.csv', index=False)
    print("데이터 저장 완료.")

In [ ]:
driver.quit()

### 4. 세부정보 제외 전체 크롤링 (1번~16번)

In [ ]:
# 데이터 저장을 위한 리스트 초기화
all_reviews = []

# 각 관광지에 대해 데이터 수집
for i in range(len(place_names)):
    try:
        # 새로 요소를 찾아 가져오기 (stale 문제 방지)
        place_elements = driver.find_elements(By.CLASS_NAME, 'tit')
        
        # n번째 관광지 요소 접근
        place_element = place_elements[i]
        
        # 관광지 요소 클릭
        place_element.click()
        time.sleep(3)  


        # 여행톡 클릭
        travel_talk_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '여행톡')))
        travel_talk_button.click()

        # 스크롤 및 댓글 더보기 클릭
        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)

                # 페이지를 약간 위로 스크롤
                driver.execute_script("window.scrollBy(0, -1000);")
                time.sleep(1)

                more_comments_button = driver.find_element(By.CSS_SELECTOR, '#commentMore > a')
                more_comments_button.click()
                time.sleep(1)

            except (NoSuchElementException, ElementNotInteractableException):
                print("더 이상 클릭할 '댓글 더보기' 버튼이 없습니다.")
                break

        # 댓글 데이터 수집
        comments = driver.find_elements(By.CSS_SELECTOR, 'div.list_reply > ul > li') 

        for comment in comments:
            try:
                author = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > em').text
                date = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > span').text
                content = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > p').text

                # 리뷰 데이터 추가
                all_reviews.append({
                    '관광지 이름': place_names[i],
                    '댓글 작성자': author,
                    '댓글 작성 시간': date,
                    '댓글': content
                })
                
            except NoSuchElementException as e:
                print(f"댓글 수집 중 오류 발생: {e}")
                continue

        # 뒤로가기
        driver.back()
        time.sleep(3)

    except Exception as e:
        print(f"오류 발생: {place_names[i]}, Error: {str(e)}")
        break

### 5. 세부정보 제외 전체 크롤링 (17번)

In [ ]:
# 데이터 저장을 위한 리스트 초기화
all_reviews = []

# 시작 인덱스 설정 (17번째 관광지부터 시작하려면 인덱스 16)
starting_index = 16

# 각 관광지에 대해 데이터 수집
for i in range(starting_index, 17):
    try:
        
        while True:
    
            try:
                # 페이지 하단으로 스크롤
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)  # 스크롤 후 로딩 시간 대기

                # 페이지를 약간 위로 스크롤
                driver.execute_script("window.scrollBy(0, -200);")  
                time.sleep(1)  
        
                # '더보기+' 버튼 찾기
                more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#contents > div > div.hotplace_sub > div.box_wrap > div.list.hotplace_list > div.more > button')))
                more_button.click()
                time.sleep(1) 
        
            except Exception as e:
                print("더 이상 로드할 관광지가 없거나 예외 발생:", e)
                break
        
        # 페이지 맨 위로 스크롤
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        
        # 새로 요소를 찾아 가져오기 (stale 문제 방지)
        place_elements = driver.find_elements(By.CLASS_NAME, 'tit')
        
        # n번째 관광지 요소 접근
        place_element = place_elements[i]
        
        # 관광지 요소 클릭
        place_element.click()
        time.sleep(3)  

        # 여행톡 클릭
        travel_talk_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '여행톡')))
        travel_talk_button.click()

        # 스크롤 및 댓글 더보기 클릭
        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)

                # 페이지를 약간 위로 스크롤
                driver.execute_script("window.scrollBy(0, -1000);")
                time.sleep(1)

                more_comments_button = driver.find_element(By.CSS_SELECTOR, '#commentMore > a')
                more_comments_button.click()
                time.sleep(1)

            except (NoSuchElementException, ElementNotInteractableException):
                print("더 이상 클릭할 '댓글 더보기' 버튼이 없습니다.")
                break

        # 댓글 데이터 수집
        comments = driver.find_elements(By.CSS_SELECTOR, 'div.list_reply > ul > li') 

        for comment in comments:
            try:
                author = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > em').text
                date = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > div.date > span').text
                content = comment.find_element(By.CSS_SELECTOR, 'div.txt_reply > p').text

                # 리뷰 데이터 추가
                all_reviews.append({
                    '관광지 이름': place_names[i],
                    '댓글 작성자': author,
                    '댓글 작성 시간': date,
                    '댓글': content
                })
                
            except NoSuchElementException as e:
                print(f"댓글 수집 중 오류 발생: {e}")
                continue
                
        # 뒤로가기
        driver.back()
        time.sleep(3)


    except Exception as e:
        print(f"오류 발생: {place_names[i]}, Error: {str(e)}")
        break

In [ ]:
# 수집한 데이터 출력
print(f"수집한 리뷰 개수: {len(all_reviews)}")
df = pd.DataFrame(all_reviews)
print(df)

# 데이터 저장
df.to_csv('17번.csv', index=False)

In [ ]:
# 드라이버 종료
driver.quit()